# **Uploading and combining data: 1**
***Complete Data***

In [1]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import statsmodels.api as sm
import scipy.stats as stats
import os
import glob
import pandas as pd
from sklearn import metrics
from sklearn.model_selection import train_test_split,cross_val_score,  GridSearchCV, KFold, StratifiedKFold, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from pprint import pprint
from matplotlib import pyplot

import random

random.seed(10)


/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# **Dataset**

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Importing all the data from the CSV

Ingesting all the data from the different CSVs 

In [3]:
import pandas as pd
import glob
path = r'drive/MyDrive/2-Business/Saturdays/ProyectoSaturdaysAI-IVEdición/dataSet' # use your path
all_files = glob.glob(path + "/*.csv")
all_files.sort()

li = []

for filename in all_files:
  df = pd.read_csv(filename, index_col=None, header=0)
  df['person'], df['status'] = os.path.splitext(filename)[0].split("/")[-1].split('_')
  li.append(df)

frame = pd.concat(li, axis=0, ignore_index=True)
frame.drop(columns=['Unnamed: 0'], inplace=True)
frame

,0_x,0_y,10_x,10_y,11_x,11_y,12_x,12_y,13_x,13_y,14_x,14_y,15_x,15_y,16_x,16_y,17_x,17_y,18_x,18_y,19_x,19_y,1_x,1_y,20_x,20_y,21_x,21_y,22_x,22_y,23_x,23_y,24_x,24_y,25_x,25_y,26_x,26_y,27_x,27_y,...,56_x,56_y,57_x,57_y,58_x,58_y,59_x,59_y,5_x,5_y,60_x,60_y,61_x,61_y,62_x,62_y,63_x,63_y,64_x,64_y,65_x,65_y,66_x,66_y,67_x,67_y,6_x,6_y,7_x,7_y,8_x,8_y,9_x,9_y,EAR,FRAME,MAR,Y,person,status
0,87.0,149.0,164.0,214.0,170.0,205.0,175.0,194.0,179.0,182.0,180.0,171.0,180.0,159.0,179.0,148.0,100.0,135.0,106.0,125.0,118.0,121.0,87.0,163.0,130.0,123.0,141.0,127.0,149.0,128.0,157.0,124.0,166.0,123.0,175.0,127.0,176.0,137.0,146.0,137.0,...,151.0,195.0,146.0,195.0,141.0,194.0,134.0,193.0,107.0,211.0,130.0,189.0,141.0,186.0,146.0,186.0,151.0,186.0,156.0,189.0,151.0,188.0,146.0,188.0,141.0,188.0,119.0,217.0,131.0,223.0,144.0,224.0,155.0,222.0,0.340278,2.0,0.422475,0,01,0
1,87.0,147.0,165.0,214.0,171.0,205.0,176.0,194.0,179.0,182.0,180.0,170.0,181.0,159.0,180.0,148.0,102.0,133.0,108.0,124.0,120.0,121.0,87.0,162.0,131.0,123.0,141.0,127.0,150.0,127.0,158.0,124.0,167.0,123.0,175.0,127.0,177.0,136.0,146.0,137.0,...,151.0,195.0,146.0,196.0,141.0,195.0,135.0,194.0,109.0,212.0,130.0,189.0,142.0,186.0,146.0,187.0,151.0,186.0,157.0,190.0,151.0,188.0,146.0,188.0,141.0,188.0,120.0,219.0,133.0,223.0,145.0,224.0,156.0,222.0,0.302029,4.0,0.424783,0,01,0
2,87.0,149.0,164.0,215.0,170.0,206.0,175.0,195.0,179.0,183.0,181.0,171.0,181.0,160.0,180.0,148.0,101.0,133.0,108.0,125.0,119.0,121.0,87.0,164.0,130.0,123.0,141.0,127.0,150.0,128.0,158.0,124.0,167.0,123.0,175.0,127.0,177.0,136.0,146.0,137.0,...,150.0,195.0,145.0,196.0,140.0,195.0,134.0,193.0,108.0,212.0,130.0,189.0,141.0,187.0,146.0,187.0,150.0,187.0,156.0,190.0,150.0,188.0,145.0,188.0,141.0,188.0,120.0,219.0,132.0,223.0,144.0,224.0,155.0,222.0,0.329248,6.0,0.438057,0,01,0
3,87.0,146.0,165.0,214.0,171.0,205.0,176.0,194.0,180.0,182.0,181.0,170.0,182.0,158.0,181.0,146.0,101.0,133.0,108.0,124.0,119.0,121.0,87.0,161.0,130.0,123.0,140.0,127.0,150.0,127.0,158.0,123.0,167.0,123.0,175.0,126.0,177.0,135.0,146.0,137.0,...,151.0,195.0,145.0,195.0,140.0,195.0,134.0,193.0,108.0,211.0,130.0,189.0,141.0,186.0,146.0,186.0,150.0,186.0,156.0,189.0,150.0,188.0,145.0,188.0,141.0,188.0,120.0,218.0,132.0,223.0,144.0,224.0,156.0,222.0,0.300374,8.0,0.437844,0,01,0
4,88.0,144.0,164.0,212.0,170.0,203.0,175.0,192.0,179.0,181.0,180.0,169.0,181.0,157.0,180.0,146.0,101.0,133.0,108.0,124.0,119.0,122.0,88.0,158.0,130.0,123.0,140.0,127.0,150.0,127.0,158.0,124.0,167.0,123.0,175.0,125.0,177.0,134.0,145.0,137.0,...,150.0,196.0,145.0,196.0,140.0,195.0,134.0,193.0,109.0,209.0,130.0,188.0,141.0,186.0,145.0,186.0,150.0,186.0,157.0,189.0,150.0,188.0,145.0,188.0,140.0,188.0,120.0,216.0,132.0,222.0,144.0,223.0,155.0,220.0,0.315413,10.0,0.453461,0,01,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
489792,192.0,127.0,269.0,203.0,280.0,194.0,289.0,183.0,294.0,169.0,297.0,154.0,299.0,139.0,300.0,124.0,199.0,107.0,204.0,98.0,214.0,95.0,192.0,141.0,224.0,97.0,233.0,101.0,247.0,99.0,257.0,95.0,268.0,93.0,279.0,95.0,286.0,102.0,241.0,113.0,...,246.0,183.0,240.0,183.0,235.0,183.0,230.0,180.0,205.0,193.0,227.0,175.0,236.0,173.0,241.0,173.0,246.0,173.0,255.0,174.0,245.0,173.0,240.0,174.0,236.0,174.0,214.0,202.0,226.0,209.0,240.0,211.0,255.0,210.0,0.277564,5992.0,0.486554,5,60,5
489793,192.0,126.0,268.0,204.0,279.0,196.0,288.0,185.0,294.0,171.0,297.0,157.0,298.0,143.0,300.0,128.0,198.0,107.0,203.0,98.0,213.0,94.0,192.0,140.0,224.0,96.0,233.0,100.0,247.0,98.0,257.0,95.0,267.0,94.0,278.0,96.0,285.0,104.0,241.0,114.0,...,246.0,183.0,240.0,184.0,235.0,183.0,230.0,181.0,204.0,191.0,227.0,175.0,236.0,173.0,240.0,173.0,245.0,173.0,255.0,174.0,245.0,174.0,240.0,174.0,236.0,174.0,213.0,200.0,225

We are going to group all the data from each CSV using the mean of each column

In [4]:
data = frame.groupby(['person', 'status']).mean().reset_index()
data

,person,status,0_x,0_y,10_x,10_y,11_x,11_y,12_x,12_y,13_x,13_y,14_x,14_y,15_x,15_y,16_x,16_y,17_x,17_y,18_x,18_y,19_x,19_y,1_x,1_y,20_x,20_y,21_x,21_y,22_x,22_y,23_x,23_y,24_x,24_y,25_x,25_y,26_x,26_y,...,55_x,55_y,56_x,56_y,57_x,57_y,58_x,58_y,59_x,59_y,5_x,5_y,60_x,60_y,61_x,61_y,62_x,62_y,63_x,63_y,64_x,64_y,65_x,65_y,66_x,66_y,67_x,67_y,6_x,6_y,7_x,7_y,8_x,8_y,9_x,9_y,EAR,FRAME,MAR,Y
0,01,0,91.322548,150.177785,171.566378,217.465310,176.980987,208.225484,180.773182,197.281521,183.459973,185.935624,184.574049,174.151101,184.505003,163.117078,183.235490,152.868579,109.911608,136.905937,117.024683,127.655771,128.954303,124.361908,91.886591,164.806204,140.789193,125.962975,151.279186,130.876918,158.782855,131.680120,166.413943,128.621081,174.414276,128.157105,181.576718,131.176117,183.138759,139.642095,...,164.428619,195.188793,160.656771,195.576718,155.963642,195.715811,150.937959,195.296531,144.524683,194.553702,113.768846,215.346231,140.357238,192.333889,151.444296,189.225817,156.242495,189.485324,160.589393,189.449633,165.014676,193.164777,160.474650,189.087392,156.142095,189.209807,151.377919,188.844897,126.647765,221.558706,140.070380,225.696798,153.274183,226.339893,163.847899,224.435624,0.370771,3002.991995,0.374084,0
1,01,10,140.583502,293.990572,251.834680,371.552862,261.477778,360.173737,268.378451,346.144444,272.055556,330.465320,272.754209,314.194613,271.379125,298.281145,268.816162,282.875421,157.254209,276.722896,164.969024,264.296633,179.329293,257.777441,142.413805,312.087542,194.794949,258.118182,209.028283,262.285522,222.677778,260.902357,233.303030,255.520202,245.089562,253.328620,256.387205,256.250842,261.387542,266.405051,...,237.747475,342.051178,231.781481,343.798990,225.271044,344.879798,218.683165,345.241414,209.880135,345.043434,174.271380,372.888889,203.838047,342.092256,217.537374,336.586532,224.032660,336.278788,230.206397,335.350842,239.412458,337.930303,230.674747,335.169697,224.461279,336.233670,218.170707,336.406734,191.273737,380.152525,208.778114,384.506061,225.720202,384.156566,240.326936,380.502357,0.291019,2993.325926,0.356404,10
2,01,5,158.817085,235.358124,289.592630,338.549749,298.469347,324.672362,305.354439,308.760469,310.073032,291.474372,312.186600,273.576884,311.755109,256.095142,309.205695,239.572529,189.788945,210.475377,203.089447,197.914238,222.026131,193.262312,159.509883,258.752094,241.086432,196.286767,257.700838,204.191625,272.027471,205.545394,283.339028,201.148409,295.359464,200.506533,305.843886,204.340704,308.747739,215.652261,...,278.834841,304.201675,273.264657,304.715913,266.050921,304.876717,258.100503,304.427136,247.346734,303.006365,198.361809,336.536348,239.837856,299.948409,258.281742,295.360804,266.055611,295.743384,273.064657,295.655946,279.936013,301.001675,273.202680,293.996315,266.231826,294.287102,258.632161,293.692797,219.824791,346.098157,241.267672,352.509883,261.948074,353.261307,278.213065,349.909883,0.336049,3014.956784,0.357525,5
3,02,0,161.904587,236.029126,277.077335,322.323401,290.325075,313.186475,299.376632,300.167392,304.338801,284.305993,306.783395,267.233010,307.800469,250.032139,307.173418,233.542015,177.416137,203.642116,186.124540,192.104118,199.822899,186.442250,162.849347,253.408437,214.902243,185.653164,230.105122,188.538668,249.950452,188.885504,264.030800,185.672246,277.996317,185.429863,290.210914,190.591229,297.223971,201.737864,...,257.515567,282.373954,249.235018,283.441915,241.970204,283.897891,234.737529,283.617677,224.948443,282.416806,186.716103,315.833278,220.180114,278.314697,234.822230,271.079009,241.534650,270.775695,248.418815,270.863408,260.354871,278.366254,248.713090,274.043522,241.841312,274.258453,234.984935,274.164379,202.351188,323.770338,220.124540,328.850017,240.876465,329.984265,261.095413,328.064279,0.307665,3013.082692,0.464897,0
4,02,10,90.521000,135.103333,152.341667,186.681667,159.175667,180.588000,164.023667,172.638000,166.657333,163.333000,168.428333,153.617000,170.253333,143.892000,171.177667,134.02533

Now we are going to create the train and test dataframes

In [5]:
x_train = data.loc[:160].drop(columns=['person', 'status', 'FRAME', 'Y'])
y_train = data.loc[:160, ['status']]
x_test = data.loc[-11:].drop(columns=['person', 'status', 'FRAME', 'Y'])
y_test = data.loc[-11:, ['status']]

Finally we create a random forest model 


In [6]:
# # Create the model with 100 trees
model = RandomForestClassifier(n_estimators=100, 
                               bootstrap = True) #verbose=2,
                               #max_features = 'sqrt')
# Fit on training data
model.fit(x_train, y_train)

# Prediction
y_pred = model.predict(x_test)

# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


Accuracy: 0.9590643274853801


In [7]:
# Get Feature Importance
importance = model.feature_importances_
importance_df = pd.DataFrame(importance, index=x_train.columns, columns=['Feature_Importance'])
importance_df = importance_df.sort_values(by='Feature_Importance', ascending=False)
importance_df.head(10)

,Feature_Importance
EAR,0.032031
MAR,0.018275
11_x,0.016087
15_x,0.015389
0_x,0.013791
10_x,0.011892
3_x,0.011855
4_x,0.011359
23_x,0.011300
17_x,0.011298


We want to discover if knowing which person the data below to is important for the model. For this we are going to mantain a column with the label of the person in the model, therefore the model could pontencially learn the features of the person... 

In [20]:
x_train = data.loc[:160].drop(columns=[ 'status', 'FRAME', 'Y'])
y_train = data.loc[:160, ['status']]
x_test = data.loc[-11:].drop(columns=[ 'status', 'FRAME', 'Y'])
y_test = data.loc[-11:, ['status']]

X_train, X_test, y_train, y_test = train_test_split(data.drop(columns=[ 'status', 'FRAME', 'Y']), data(['status'], test_size=0.2, random_state=42)

In [21]:
# # Create the model with 100 trees
model = RandomForestClassifier(n_estimators=1000, 
                               bootstrap = True) #verbose=2,
                               #max_features = 'sqrt')
# Fit on training data
model.fit(x_train, y_train)

# Prediction
y_pred = model.predict(x_test)

# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


Accuracy: 0.9590643274853801


In [19]:
# Get Feature Importance
importance = model.feature_importances_
importance_df = pd.DataFrame(importance, index=x_train.columns, columns=['Feature_Importance'])
importance_df = importance_df.sort_values(by='Feature_Importance', ascending=False)
importance_df.head(20)

,Feature_Importance
EAR,0.034085
MAR,0.014027
16_x,0.013035
13_x,0.012717
15_x,0.012038
25_x,0.011999
14_x,0.011858
12_x,0.011688
11_x,0.011459
26_x,0.011213


The result of the experiment is that person is in the top 20 of importance, but it's not super relevant. 